<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:30px;color:white"><u>Homework 4</u>    
<p style="font-family:cursive;font-size:17px; color:white" > 
<a style="font-family:cursive;font-size:17px; color: yellow" href="https://datatalks.club/courses/2021-winter-ml-zoomcamp.html" target="_blank"> <u>Link for the Course</u></a></p>
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#1">A. Preapartion for assignment</a></li>    
<ul>
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#1.1">Importing Data</a></li>   
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#1.2">2. Code to de-code encoded numbers</a></li> 
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#1.3">3. Prepare the numerical variables </a></li>
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#1.4"> 4. Remove clients with unknown default status  </a></li> 
<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#1.5"> 5. Create the target variable  </a></li>  
    </ul>

<li style = "line-height: 0.7"><a style="font-family:cursive;font-size:17px; color:#ecfe15" href = "#2"> B.  Questions</a></li>    
</div>    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, auc
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import recall_score, precision_score

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv

In [ ]:
df = pd.read_csv('CreditScoring.csv')
df.columns = df.columns.str.lower()

In [ ]:
status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)


home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

In [ ]:
for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=0)
df = df[df.status != 'unk'].reset_index(drop=True)
df['default'] = (df.status == 'default').astype(int)
del df['status']






<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Your code
<ul style="font-family:cursive;font-size:15px;color:  white">
    <li>What are the categorical variables? What are the numerical?</li>    
</ul> 
</div>



In [ ]:
categorical = [col for col in df.columns if df[col].dtypes == 'object']
numerical = [col for col in df.columns if col not in categorical]
print("Numerical Columns: ", numerical)
print("Categorical Columns: ", categorical)


<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Task
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. Use train_test_split funciton for that with random_state=1</li>
 
</ul> 
</div>


In [ ]:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)
df_train, df_valid = train_test_split(df_full_train, test_size = 0.25, random_state = 1)

In [ ]:
len(df_train) / len(df)

<div style="background:#2b6684;color:white; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 1:
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>ROC AUC could also be used to evaluate feature importance of numerical variables. Let's do that</li>
<ul>
<li>For each numerical variable, use it as score and compute AUC with the "default" variable</li> 
<li> Use the training dataset for that</li> 
    
   
</ul> 
<li>If your AUC is greater than 0.5, invert this variable by putting "-" in front</li>   
<li>(e.g. -df_train['expenses'])</li> 
<li>AUC can go below 0.5 if the variable is negatively correlated with the target varialble. You can change the direction of the correlation by negating this variable - then negative correlation becomes positive.If your AUC is greater than 0.5, invert this variable by putting "-" in front</li>      
</ul> 
</div>

In [ ]:
for col in numerical:
    auc_score = roc_auc_score(df_train.default, df_train[col])
    if auc_score < 0.5:
        auc_score = roc_auc_score(df_train.default, -df_train[col])
    print(f'{col: <10}: {auc_score: .4f}')   

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Training the model. From now on, use these columns only: ['seniority', 'income', 'assets', 'records', 'job', 'home']
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:</li>
<li>LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)</li>

</ul> 
</div>

In [ ]:
features = ['seniority', 'income', 'assets', 'records', 'job', 'home']

"""one-hot encoding datasets"""
dv = DictVectorizer()
train_dict = df_train[features].to_dict(orient = 'records')
val_dict = df_valid[features].to_dict(orient = 'records')

"""Creating X_train, y_train, """
X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)
y_train = df_train['default'].values
y_valid = df_valid['default'].values

"""Fitting the Model on Training Set"""
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)
predictions = model.predict_proba(X_val)[:, 1]

"""AUC for this model"""
auc_score = roc_auc_score(y_valid, predictions)
print("Answer 2: AUC of the model %s" %round(auc_score, 3))

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 3
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Now let's compute precision and recall for our model.</li>
<ul>
<li>Evaluate the model on all thresholds from 0.0 to 1.0 with step 0.01</li>
<li>For each threshold, compute precision and recall</li>
<li>Plot them</li>
</ul>
</ul> 
</div>

In [ ]:
# function to get Precision and Recall scores for a given range of thresholds

def precision_recall_dataframe(y_val, y_pred, thresholds):
    scores = []

    for t in thresholds:
        actual_positive = (y_val == 1)
        actual_negative = (y_val == 0)

        predict_positive = (y_pred >= t)
        predict_negative = (y_pred < t)

        tp = (predict_positive & actual_positive).sum()
        tn = (predict_negative & actual_negative).sum()

        fp = (predict_positive & actual_negative).sum()
        fn = (predict_negative & actual_positive).sum()

        scores.append((t, tp, fp, fn, tn))

    columns = ['threshold', 'tp', 'fp', 'fn', 'tn']
    df_scores = pd.DataFrame(scores, columns=columns)

    df_scores['precision'] = df_scores.tp / (df_scores.tp + df_scores.fp)  
    df_scores['recall'] = df_scores.tp / (df_scores.tp + df_scores.fn) 
        
        
    df_scores['tpr'] = df_scores.tp / (df_scores.tp + df_scores.fn)
    df_scores['fpr'] = df_scores.fp / (df_scores.fp + df_scores.tn)
    return df_scores[['threshold','precision', 'recall']]

"""Calculating Precision and Recall for various thresholds"""
th = np.arange(0, 1.01, 0.01)
df_scores = precision_recall_dataframe(y_valid, predictions, th)

"""Plotting the data"""
plt.plot(df_scores[['precision', 'recall']])
plt.xlabel('Threshold')
plt.title('Precision-Recall curves')
plt.grid()


<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 4
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Precision and recall are conflicting - when one grows, the other goes down. That's why they are often combined into the F1 score - a metrics that takes into account both.</li>
<ul>
<li>This is the formula for computing F1:</li>
<li>$$F_1 = 2 \cdot \cfrac{P \cdot R}{P + R}$$</li>
<li>Where $P$ is precision and $R$ is recall.</li>
<li>Let's compute F1 for all thresholds from 0.0 to 1.0 with increment 0.01</li>
</ul>
</ul> 
</div>

In [ ]:
df_scores['f1'] = 2 * (df_scores.precision * df_scores.recall)/(df_scores.precision + df_scores.recall)
df_scores[df_scores['f1'] == max(df_scores['f1'])]

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 5
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Use the KFold class from Scikit-Learn to evaluate our model on 5 different folds:</li>
<ul>
<li>KFold(n_splits=5, shuffle=True, random_state=1)</li>
<li>Iterate over different folds of df_full_train</li>
<li>Split the data into train and validation</li>
<li>Train the model on train with these parameters: LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)</li>
<li>Use AUC to evaluate the model on validation</li>
</ul>
</ul> 
</div>

In [ ]:
from sklearn.model_selection import KFold

scores = []

"""Using KFold from sklearn"""
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

"""Iterate over different folds of full_train"""
for fold, (train_idx, val_idx) in enumerate(kfold.split(df_full_train)):
        
        """Split the data into train and validation"""
        df_train = df_full_train.iloc[train_idx]
        df_valid = df_full_train.iloc[val_idx]

        y_train = df_train.default.values
        y_valid = df_valid.default.values
        
        """Apply one-hot-encoding using DictVectorizer"""
        train_dict = df_train[features].to_dict(orient='records')
        val_dict = df_valid[features].to_dict(orient='records')
        
        dv = DictVectorizer(sparse=False)

        X_train = dv.fit_transform(train_dict)
        X_val = dv.transform(val_dict)
        
        # Train Logistic regression on train part
        model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
        model.fit(X_train, y_train)

        # Use AUC to evaluate the model on validation part
        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_valid, y_pred)
        
        # save fold score
        scores.append(auc)

        print('fold %s roc-auc score: %s' %(fold, auc))

print("\n Standard Deviation of Scores: %s" %np.std(scores).round(3))        

<div style="background:#2b6684; font-family:cursive;padding:0.5em;border-radius:0.2em;font-size:20px;color:yellow">Question 6
<ul style="font-family:cursive;font-size:15px;color:  white">
<li>Now let's use 5-Fold cross-validation to find the best parameter C</li>
<ul>
<li>Iterate over the following C values: [0.01, 0.1, 1, 10]</li>
<li>Use these parametes for the model: LogisticRegression(solver='liblinear', C=C, max_iter=1000)</li>
<li>Compute the mean score as well as the std</li>
</ul>
</ul> 
</div>

In [ ]:
scores = list()
"""C Values"""
C_values = [0.01, 0.1, 1, 10]

kfold = KFold(n_splits=5, shuffle=True, random_state = 1)

"""Iterate over the C values"""
for C in C_values:
    kfold_scores = list()
    """Iterate over different folds of df_full_train"""
    for train_idx, val_idx in kfold.split(df_full_train):
            
            df_train = df_full_train.iloc[train_idx, :]
            df_val = df_full_train.iloc[val_idx, :]

            y_train = df_train.default.values
            y_val = df_val.default.values

            train_dict = df_train[features].to_dict(orient='records')
            val_dict = df_val[features].to_dict(orient='records')
        
            dv = DictVectorizer(sparse=False)
        
            X_train = dv.fit_transform(train_dict)
            X_val = dv.transform(val_dict)

            model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
            model.fit(X_train, y_train)

            y_pred = model.predict_proba(X_val)[:, 1]
            auc = roc_auc_score(y_val, y_pred)
            
            kfold_scores.append(auc)
    
    """compute the mean and std of scores"""
    mean_auc = np.mean(kfold_scores)
    std = np.std(kfold_scores)
    
    """save mean auc and std of current C value"""
    kfold_scores.extend([mean_auc, std])
    scores.append(kfold_scores)
    
    print(f'C={C:<5} average roc-auc: {mean_auc:.4f} +/- {std:.3f}')

In [ ]:
df_AUC = pd.DataFrame(scores, columns = ['fold_1', 'fold_2', 'fold_3', 'fold_4', 
                                        'fold_5', 'mean_auc', 'std'], 
            index=[str(c) for c in C_values]).rename_axis('C')

df_AUC[df_AUC['mean_auc'] == max(df_AUC['mean_auc'])].reset_index()